## Scan for Yaml files:

In [ ]:
from pathlib import Path

benefits_folder = Path('./aides')

def extract_benefits_paths(benefits_folder):
    isYAMLfile = lambda path : str(path).endswith('.yml') or str(path).endswith('.yaml')
    liste_fichiers = [str(benefit) for benefit in benefits_folder.iterdir() if isYAMLfile(benefit)]
    return liste_fichiers

benefits_files_paths = extract_benefits_paths(benefits_folder)
benefits_files_paths

## Open and parse files: make a list of Dict

In [ ]:
import yaml

def extract_benefit_file_content(aide_path):
    return yaml.safe_load(open(aide_path))

def extract_benefits_from_yamls(path):
    benefits_files_paths = extract_benefits_paths(path)
    benefits = [extract_benefit_file_content(path) for path in benefits_files_paths]
    return benefits

In [ ]:
class Benefit():
    label = str
    institution = str
    description = str
    prefix = str
    conditions_generales: list
    profils: list
    conditions: list
    interestFlag: str
    type: str
    unit: str
    periodicite: str
    montant: int
    link: str
    instructions: str
    teleservice: str
    periodicite: str
    legend: str

    def __init__(self, dict: dict) -> None:

        try:
            self.label = dict.get('label', '')
            self.institution = dict.get('institution', '')
            self.description = dict.get('description', '')
            self.prefix = dict.get('prefix', '')
            self.conditions_generales = dict.get('conditions_generales', [])
            self.profils = dict.get('profils', [])
            self.conditions = dict.get('conditions', [''])
            self.interestFlag = dict.get('interestFlag', '')
            self.type = dict.get('type', '')
            self.unit = dict.get('unit', '')
            self.periodicite = dict.get('periodicite', '')
            self.montant = dict.get('montant', None)
            self.link = dict.get('link', '')
            self.instructions = dict.get('instructions', '')
            self.teleservice = dict.get('teleservice', '')
            self.periodicite = dict.get('periodicite', '')
            self.legend = dict.get('legend', '')
        except KeyError:
            raise KeyError


In [ ]:
from benefit import Benefit
def parse_benefits_from_yamls(path):
    benefits_files_paths = extract_benefits_paths(path)
    benefits = [Benefit(extract_benefit_file_content(path)) for path in benefits_files_paths]
    return benefits

benefits_objects = parse_benefits_from_yamls(benefits_folder)    

In [ ]:
benefits = extract_benefits_from_yamls(benefits_folder)

##  Make a list of [profils] [type] in all aides given

In [ ]:

from functools import reduce
from benefit import BenefitProfile

def accumulate_profils(profils: list, benefit: BenefitProfile):
        return profils + benefit.get_profils_types()

profiles_names = reduce(accumulate_profils, benefits_objects, [])


# Make a list with all conditons:
	- conditions generales
	- conditions in [profils][type]

In [ ]:
from functools import reduce

from benefit import BenefitCondition

def accumulate_conditions(conditions: list, benefit: BenefitCondition):
    return conditions + benefit.get_all_conditions_types()

conditions_names = reduce(accumulate_conditions, benefits_objects, [])
conditions_names

# Count frequence of profils and conditions appearances

In [ ]:
def count_list_item_frequence(list : list):
    elements_counter = {}
    for element in list:
        if element in elements_counter:
            elements_counter[element] +=1
        else:
            elements_counter[element] = 1
    return (sorted(elements_counter.items(), key=lambda x:x[1]))

profil_names_frequence = count_list_item_frequence(profiles_names)
condition_frequence = count_list_item_frequence(conditions_names)

In [ ]:
condition_frequence

In [ ]:
profil_names_frequence

## Reference benefits with no profils:

In [ ]:
def contains_profils(benefit :dict):
    return 'profils' in benefit and benefit['profils'] != []

def extract_benefit_without_profile(benefit: Benefit):
    if benefit.profils == []:
        return [benefit.get_label_and_institution()]
    else:
        return []

def accumulate_benefit_without_profile(list:list, benefit):
    
    return list + extract_benefit_without_profile(benefit)
    
    
benefits_without_profils = reduce(accumulate_benefit_without_profile, benefits_objects, [])
benefits_without_profils

## Benefits using only age as condition:

In [ ]:
def is_only_condition_age(benefit: Benefit):
    conditions = benefit.get_all_conditions_types()
    return all(condition == "age" for condition in conditions) and conditions != []

def extract_benefit_with_only_condition(benefit : Benefit):
    if is_only_condition_age(benefit):
        return [benefit.get_label_and_institution()]
    else:
        return []

def accumulate_benefit_with_only_condition(list:list, benefit: Benefit):
    
    return list + extract_benefit_with_only_condition(benefit)

benefits_with_only_age_condition = reduce(accumulate_benefit_with_only_condition, benefits_objects, [])
benefits_with_only_age_condition